In [6]:
import requests_with_caching

In [7]:
apikey = "abcd1234"  # you may *optionally* replace this with your API key.
# Note: you do *not* need an API key to complete this assignment. Every request should be in the cache
requests_with_caching.clear_cache()
# print(list(requests_with_caching.perm_cache().keys()))

In [8]:
# NOTE: the OMDb API uses http:// instead of https://
import json
def get_movie_data(name: str) -> dict:
    param = {}
    param["t"] = name
    param["r"] = 'json'
    this_page_cache = requests_with_caching.get("http://www.omdbapi.com/", params = param)
    response = json.loads(this_page_cache.text)
    return response


In [9]:
results = get_movie_data("Black Panther")
print(results)
assert type(results) == type({})
assert results["Year"] == "2018"
print(json.dumps(results, indent = 4))
# some other invocations that we use in the automated tests; uncomment these if you are getting errors and want better error messages
# print(get_movie_data("Venom"))
# print(get_movie_data("Baby Mama"))

found in permanent_cache
{'Title': 'Black Panther', 'Year': '2018', 'Rated': 'PG-13', 'Released': '16 Feb 2018', 'Runtime': '134 min', 'Genre': 'Action, Adventure, Sci-Fi', 'Director': 'Ryan Coogler', 'Writer': 'Ryan Coogler, Joe Robert Cole, Stan Lee', 'Actors': "Chadwick Boseman, Michael B. Jordan, Lupita Nyong'o", 'Plot': "T'Challa, heir to the hidden but advanced kingdom of Wakanda, must step forward to lead his people into a new future and must confront a challenger from his country's past.", 'Language': 'English, Swahili, Nama, Xhosa, Korean', 'Country': 'United States', 'Awards': 'Won 3 Oscars. 123 wins & 289 nominations total', 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTg1MTY2MjYzNV5BMl5BanBnXkFtZTgwMTc4NTMwNDI@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.3/10'}, {'Source': 'Rotten Tomatoes', 'Value': '96%'}, {'Source': 'Metacritic', 'Value': '88/100'}], 'Metascore': '88', 'imdbRating': '7.3', 'imdbVotes': '837,583', 'imdbID': 'tt1825

In [10]:
def rt_rating(movie_data: dict) -> int:
    rating = ""
    for rating_list in movie_data["Ratings"]:
        if rating_list["Source"]== "Rotten Tomatoes":
            rating = rating_list["Value"]
    if rating != "":
        int_rating = int(rating[:2])
    else: int_rating = -1
    return int_rating
            




#We suggest that you write an assert statement to check the output of your function for the movie Black Panther. The autograder will check results for some other movies.
results = get_movie_data("Black Panther")
assert rt_rating(results) == 96
print(rt_rating(results))

found in permanent_cache
96


In [11]:
def get_joke_data(name : str) -> dict:
    url = "https://icanhazdadjoke.com/search"
    param = {}
    param["term"] = name
    param["limit"] = 2
    response = requests_with_caching.get(url, params = param)
    json_response = json.loads(response.text)
    return json_response

In [12]:
assert (
    len(get_joke_data("magic")["results"]) == 2
), "The correct number of jokes for 'magic' should be 2"
assert (
    get_joke_data("magic")["results"][0]["joke"]
    == "What do you call a magician who has lost their magic? Ian."
)

found in permanent_cache
found in permanent_cache


In [13]:
def get_jokes(plot: str, verbosity=0) -> tuple[str, list[str]]:
    """Returns a tuple containing the longest word for which jokes were found
    and the joke itself. Break ties for longest word using the order in `plot`.
    Make sure that you strip punctuation from the word before you search for a joke.

    Parameters
    ----------
    plot : str
        The plot of a movie.

    verbosity : int (optional)
        If 0, no output is printed. If 1, some output is printed about which words were tried.
        Defaults to 0.

    Returns
    -------
    tuple[str, list[str]]
        A tuple containing the word that was used to search for a joke and a list of two joke strings.
    """

    words = plot.split()  # split into separate words
    words = [w.strip(",.!;:") for w in words]  # remove punctuation for each word
    words.sort(key=lambda w: (-len(w), plot.index(w)))

    # Step 3: Iterate through sorted words and find jokes
    for word in words:
        if verbosity == 1:
            print(f"Trying word: {word}")

        # Fetch jokes for the current word
        joke_data = get_joke_data(word)
        
        # If jokes are found, return the word and the list of jokes
        if joke_data['results']:
            jokes = [result['joke'] for result in joke_data['results']]
            return (word, jokes)

    # Step 4: If no jokes are found, return (None, None)
    return (None, None)

    # WRITE YOUR CODE HERE
plot = "I had dreams of a cat."
result = get_jokes(plot)
print(result)


found in permanent_cache
('dreams', ["I'm tired of following my dreams. I'm just going to ask them where they are going and meet up with them later."])


In [14]:
# Test case 1: Plot with a word that has jokes
plot = "I had dreams of a cat."
result = get_jokes(plot, 1)
print(result)
assert result[0] == "dreams"
assert (
    result[1][0]
    == "I'm tired of following my dreams. I'm just going to ask them where they are going and meet up with them later."
)

Trying word: dreams
found in permanent_cache
('dreams', ["I'm tired of following my dreams. I'm just going to ask them where they are going and meet up with them later."])


In [15]:
def highlight(word: str, sentence: str) -> str:
    """
    Highlights a specific word in a sentence by surrounding it with asterisks (**).
    The highlighting is case-insensitive.

    Args:
        word (str): The word to be highlighted.
        sentence (str): The sentence in which the word should be highlighted.

    Returns:
        str: The sentence with the specified word highlighted.
    """
    import re

    # Escape special characters in the word to treat it as a literal string
    # Use re.sub() to replace the word with the highlighted version
    # Use re.IGNORECASE flag to perform case-insensitive replacement
    return re.sub(re.escape(word), f"**{word}**", sentence, flags=re.IGNORECASE)

In [16]:
def haha_me(movie_title: str, verbosity=0) -> str:
    #name of the movie
    movie_name = movie_title
    
    #rotten tomatoes rating
    r = get_movie_data(movie_title)
    t = r["Ratings"][1]["Value"]
    rate = f"Rotten Tomatoes rating: {t}"
    
    #plot
    plot = r["Plot"]
    #highlighting the word
    highlighted_word = get_jokes(plot)[0]
    highlighted_plot = highlight(highlighted_word, plot)
    
    #next line
    highlight1 = f"**{highlighted_word}**"
    nxt_line = f"Speaking of {highlight1}, that reminds me of some jokes."
    
    #conditional phrasing
    conditional_statement = ""
    ratings = rt_rating(r)
    if ratings == -1:
        conditional_statement = "Hope you like them!"
    elif ratings < 70:
        conditional_statement = "Hope they're better than the movie!"
    else:
        conditional_statement = "Hope they're as good as the movie!"
        
    #list of jokes seperated by a newline 
    jokes = get_jokes(plot)
    jokes = jokes[1]
    jokes = "\n".join(jokes)
    jokes = highlight(highlighted_word, jokes)
    
    final_str = f"""{movie_name}
{rate}
{highlighted_plot}
{nxt_line}
{conditional_statement}

{jokes}"""
    
    return final_str
haha_me("Black Panther")

found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache


"Black Panther\nRotten Tomatoes rating: 96%\nT'Challa, heir to the hidden but advanced kingdom of Wakanda, must step **forward** to lead his people into a new future and must confront a challenger from his country's past.\nSpeaking of **forward**, that reminds me of some jokes.\nHope they're as good as the movie!\n\nSometimes I tuck my knees into my chest and lean **forward**.  That’s just how I roll.\nWhy do scuba divers fall backwards into the water? Because if they fell **forward**s they’d still be in the boat."

In [17]:
assert(
    haha_me("Black Panther")
    == """Black Panther
Rotten Tomatoes rating: 96%
T'Challa, heir to the hidden but advanced kingdom of Wakanda, must step **forward** to lead his people into a new future and must confront a challenger from his country's past.
Speaking of **forward**, that reminds me of some jokes.
Hope they're as good as the movie!

Sometimes I tuck my knees into my chest and lean **forward**.  That’s just how I roll.
Why do scuba divers fall backwards into the water? Because if they fell **forward**s they’d still be in the boat."""
)

found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
